In [3]:
using Combinatorics
using CUDA
using DelimitedFiles
using IterTools
using OrderedCollections
using Printf
using Random
using ResumableFunctions
using StatsFuns

# This allows adapting CPU structures to be used by the GPU in CUDA
# just grab a struct and issue the command: Adapt.@adapt_structure struct_name
# 
#import Adapt

# load modules that we'll be needing only if run as a jupyter notebook
# Notice that the same sentence allows us knowing what modules have been
# loaded: isdefined(Main, :Statistics) returns 'true' in this case
# 
if  isdefined(Main, :IJulia) 
    using PyPlot
end

In [4]:
# include the path to user-defined modules
# 
# computer is for remotely mounted directories. Otherwise leave it blank
#
# computer = "/colilo"
# computer = ""

gethostname() == "colilo" ? computer = "" : computer = "/colilo"

push!(LOAD_PATH,homedir()*computer*"/Julia_1/Modules");
push!(LOAD_PATH,homedir()*computer*"/Julia_1/Modules/RBM");
push!(LOAD_PATH,homedir()*computer*"/Julia_1/Modules/CUDA");
push!(LOAD_PATH,homedir()*computer*"/Julia_1/Modules/Tests");

In [5]:
using RBM_module_J1
using RBM_module_CUDA_J1
using Writing_module

#Adapt.@adapt_structure RBM_net

In [21]:
INP            = OrderedDict()
INP["T0"]      = 1000.0
INP["Nβ"]      = 1000
INP["λ"]       = 0.92
INP["Nepochs"] = 100

INP["IOUT"]    = stdout

IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(44) open, 0 bytes waiting)))

In [22]:
Parallel_Temp(INP)

# Tempering temperatures : 


1×1000 adjoint(::Vector{Float64}) with eltype Float64:
 1000.0  920.0  846.4  778.688  716.393  659.082  …  7.24856e-34  6.66867e-34

LoadError: suputamadre!

In [ ]:
function Parallel_Temp(dict_INP::Union{Dict,OrderedDict})
    
# Read the data from the dictionary and assign values to local variables
# with the same key names
# This fills all data we have in dictionary dict_INP    
#     
#    for kv in dict_INP
#        valor         = kv[2]
#        nom           = Symbol(kv[1])
#        @eval (($nom) = ($valor))
#    end
    
# Parameters
# 
    T0 = dict_INP["T0"]
    Nβ = dict_INP["Nβ"]
    λ  = dict_INP["λ"]
    
# Output management
# 
#   dict_INP["IOUT"] != stdout ? IOUT = open(dict_INP["IOUT"],"w") : IOUT = stdout
    
# Build the array of temperatures
#     
    T                 = zeros(Nβ)

# geometric temperature tempering scheme    
#     
    T[1]              = T0
    T[2:end]          = [ T[i] = T[i-1]*λ for i in 2:Nβ ]
   
# linear temperature tempering scheme    
#     
#    T         = [ T0*(1.0 - 0.9999*(i-1)/(Nβ-1)) for i in 1:Nβ]
       
# Print the list of tempering Temperatures
#     
    println(IOUT,"# Tempering temperatures : ")
    if  dict_INP["IOUT"] == stdout
        display(T')
    else
        for temp in T
            @printf IOUT "%14.6g  " temp
        end
        println(IOUT,"")
    end    
    error("suputamadre!")
    
# convert the temperatures array to CUDA
#     
    Tc        = CuArray(T')
    
# Read weights and bias from the input dictionary
# also the number of visible and hidden units    
#     
    B         = dict_INP["B"]
    Ct        = dict_INP["Ct"]
    Wij       = dict_INP["Wij"]
    
    Nv, Nh    = size(Wij)
    
# Initial states -must be an array of size Nh·Nβ corresponding
# to Nβ column vectors of Nh components each    
#     
    uo        = dict_INP["u_ini"]
    size(uo) != (Nh,Nβ) ? error("NO MATCHING INITIAL STATE u_ini.\nSTOP.") : 0
    
# Starting state and value of the function    
# 
    uc          = CuArray(uo)
    Bo          = Bounds_logZ_CUDA(uc,B,Ct,Wij)
    Bmax, CImax = findmax(Bo)
    umax        = uo[:,CImax[2]]

    println(IOUT,"# \n# Start the Tempering algorithm \n# ")
    
# create the list of best values found and the list of the corresponding states
# we arrange states in list_ubest by columns    
#    
    list_best          = zeros(Nbest+1)
    list_best[1:Nbest] = sort(Bo[1:Nbest],rev=true)    
    list_ubest         = falses(Nh,Nbest+1)
    
# Now DO the ITERATIONS
# 
    for iswaps in 1:Nepochs
        for iMC in 1:NMC
            
            # ONE MC step flipping Nflip units of each column vector, corresponding to each temperature
            # 
            # Array of units to change proposal
            # This is a Nflip·Nβ array of integers in 1:Nh, as per columns
            # you have the proposed units to change at each temperature of the
            # tempering scheme           
            #             
            idx              = rand(1:Nh,Nflip,Nβ) # NOTE: choose a number from 1 to Nh NflipxNβ times. This is to choose the unit
            # NOTE: idx is then a matrix of Nflip rows and Nβ columns

            # idx indicates which units to flip:
            # EXAMPLE:
            # if idx[1,1] = 3, means the third unit of the first chain is to be flipped. The other 1 indicates it is the first flip
            # if idx[2,1] = 5, means the fifth unit of the first chain is to be flipped. The 2 indicates it is the second flip

            # build the proposed new states with the previous changes
            #             
            un               = copy(uo) 
            [un[idx[:,i],i] .= .!un[idx[:,i],i] for i in 1:Nβ] # translated: the new un is the flip (! function ie a not) of the selected indices by the idx matrix. 
            # On the right we have the not function applied to the un matrix at the selected indices
            
            # evaluate the Bounds function value of the new states
            #             
            uc               = CuArray(un)
            Bn               = Bounds_logZ_CUDA(uc,B,Ct,Wij)
            
            # Evaluate the Bounds function for the new states, and keep the best ones
            # in case they are better than the ones already stored            
            #             
            Bmax2, idx_max   = findmax(vec(Bn))
            if  Bmax2 > Bmax
                Bmax  = Bmax2
                umax  = un[:,idx_max]
            end
            
            un_id_max                  = un[:,idx_max]
            diff_best                  = abs.(list_ubest .- un_id_max)
            s_diff_best                = sum(diff_best,dims=1)
            if sum(s_diff_best .== 0) == 0
                list_best[end]         = Bmax2
                idx_best               = sortperm(list_best,rev=true)
                list_best[1:Nbest]     = list_best[idx_best[1:Nbest]]
                list_ubest[:,end]     .= un[:,idx_max]
                list_ubest[:,1:Nbest] .= list_ubest[:,idx_best[1:Nbest]]               
            end
            
            # Metropolis accepatance/rejection
            # changes to accept
            #             
            idx              = Array(exp.((Bn .- Bo) ./ Tc) .>= CUDA.rand(1,Nβ)) # indices of the temperatures that accepted a change
            uo[:,vec(idx)]  .= un[:,vec(idx)] # initial state is updated with the old accepted state
            Bo[idx]         .= Bn[idx] # the old bounds are updated with the new bounds
        end # do this NMC times
        
        # Temperature swaps
        #         
        for i in 1:Nβ-1
            if  exp(Bo[i]/Tc[i+1] + Bo[i+1]/T[i] - (Bo[i]/Tc[i] + Bo[i+1]/T[i+1])) > rand()
                Bo[i]   , Bo[i+1]    = Bo[i+1] , Bo[i]
                aux       = copy(uo[:,i])
                uo[:,i]   = copy(uo[:,i+1])
                uo[:,i+1] = aux
            end
        end
        
        end
        
    end
    
    dict_INP["List_Best"]  = list_best[1:end-1]
    dict_INP["List_uBest"] = list_ubest[:,1:end-1]
    dict_INP["umax"]       = umax
    dict_INP["Bmax"]       = Bmax
    
    println(IOUT,"# \n")
    println(IOUT,"# Best value found : $(Bmax)\n")
    println(IOUT,"# Corresponding to the state\n")
    for elem in umax
        elem ? print(IOUT,1," ") : print(IOUT,0," ")
    end
    println(IOUT,"")
    
# clouse the output file if required
# 
    dict_INP["IOUT"] != stdout ? close(IOUT) : 0
    
end;